In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import yfinance as yf

In [2]:
def get_stock_code_list_bursa():
    url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find(id = "MainContent2_tbAllStock")
    stock_list = table.find_all('a')
    return [stock_code.get('href')[-4:] for stock_code in stock_list]


def get_stock_name_list_bursa():
    url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find(id = "MainContent2_tbAllStock")
    stock_list = table.find_all('a')
    return [stock_code.getText() for stock_code in stock_list]

In [3]:
def get_stock_price(ticker):
    data = yf.download(ticker, start="2021-01-01")
    return data


def add_EMA(price, day):
    return price.ewm(span=day).mean()


def add_STOCH(close, low, high, period, k, d=0): 
    STOCH_K = ((close - low.rolling(window=period).min()) / (high.rolling(window=period).max() - low.rolling(window=period).min())) * 100
    STOCH_K = STOCH_K.rolling(window=k).mean()
    if d == 0:
        return STOCH_K
    else:
        STOCH_D = STOCH_K.rolling(window=d).mean()
        return STOCH_D

In [4]:
!pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=59878801a96949148489dc77682ff59e89f746b6b5b4f23ef0e5b5cc292aab42
  Stored in directory: c:\users\zac\appdata\local\pip\cache\wheels\c4\71\30\9b87e7cc8350a4226ffbed6e4a6a923ac720845398895eb35d
Successfully built ta


In [5]:
from ta.momentum import RSIIndicator

In [15]:
#Condition 1: Overall trend is uptrend 
#Condition 2: RSI must be lower than 70, to prevent overbuy and buy high sell low
#Condition 3: Volume must be higher than 200k to prevent low liquidity 
#Condition 4: Close must higher than Open 
#Condition 5: Closing price must be higher than previous closing price 
#Condition 6: Candle Stick must be on EMA 25 
#Condition 7: Price difference must be less than 6% (Close - Open)
#Condition 8: Price different between opening price and EMA 25 must be less than 4%

def check_condition(df):
    candle1 = df.iloc[-1]
    candle2 = df.iloc[-2]
    
    cond1 = candle1['EMA25'] > candle1['EMA50'] > candle1['EMA100']
    cond2 = candle1['STOCH_%K(5,3,3)'] <= 70 or candle1['STOCH_%D(5,3,3)'] <= 70
    cond3 = candle1['Volume'] > 200000
    cond4 = candle1['Close'] > candle1['Open']
    cond5 = candle1['Close'] > candle2['Close']
    cond6 = candle1['Open'] > candle1['EMA50'] and candle1['Close'] > candle1['EMA25']
    cond7 = (candle1['Close'] - candle1['Open']) / candle1['Open'] <= 0.06
    cond8 = (candle1['Open']- candle1['EMA25']) / candle1['Open'] <= 0.04
    
    return cond1 and cond2 and cond3 and cond4 and cond5 and cond6 and cond7 and cond8


In [5]:
screened_list = [] 

stock_list = get_stock_code_list_bursa()

In [13]:
stock_list

['0012',
 '7054',
 '5238',
 '00EA',
 '7086',
 '7131',
 '7120',
 '7191',
 '9148',
 '7146',
 '6599',
 '5139',
 '5185',
 '2488',
 '5198',
 '7315',
 '7090',
 '0122',
 '5099',
 '5014',
 '2682',
 '2658',
 '7609',
 '9954',
 '5115',
 '5116',
 '2674',
 '1163',
 '1015',
 '0159',
 '5120',
 '7031',
 '6351',
 '7083',
 '4758',
 '0048',
 '6556',
 '5568',
 '5088',
 '5015',
 '6432',
 '7214',
 '7007',
 '5210',
 '5127',
 '1481',
 '0068',
 '0150',
 '7722',
 '0039',
 '7129',
 '4057',
 '0105',
 '7162',
 '6399',
 '7070',
 '0072',
 '7048',
 '5130',
 '7181',
 '7579',
 '6888',
 '5106',
 '5021',
 '7078',
 '0098',
 '7251',
 '4162',
 '6602',
 '6173',
 '5190',
 '9814',
 '8133',
 '7241',
 '7005',
 '5258',
 '6998',
 '7243',
 '5032',
 '3239',
 '5248',
 '3395',
 '5196',
 '4219',
 '1562',
 '1899',
 '7187',
 '5069',
 '0168',
 '9288',
 '7036',
 '6297',
 '5100',
 '5932',
 '9474',
 '8761',
 '9938',
 '7221',
 '2771',
 '0011',
 '7188',
 '1818',
 '7174',
 '7154',
 '7128',
 '5105',
 '5229',
 '0163',
 '2836',
 '0173',
 '7076',
 

In [8]:
stock_name = get_stock_name_list_bursa()

In [9]:
stock_name

['3A',
 'AASIA',
 'AAX',
 'ABFMY1',
 'ABLEGRP',
 'ACME',
 'AXTERIA',
 'ADVENTA',
 'ADVPKG',
 'AEM',
 'AEON',
 'AEONCR',
 'AFFIN',
 'ABMB',
 'AFUJIYA',
 'AHB',
 'AHEALTH',
 'AIM',
 'CAPITALA',
 'AIRPORT',
 'PARKWD',
 'AJI',
 'AJIYA',
 'RGTBHD',
 'ALAM',
 'ALAQAR',
 'ALCOM',
 'ALLIANZ',
 'AMBANK',
 'AMEDIA',
 'AMFIRST',
 'AMTEL',
 'AMWAY',
 'ANALABS',
 'ANCOM',
 'ANCOMLB',
 'ANNJOO',
 'APB',
 'APEX',
 'APM',
 'APOLLO',
 'ARANK',
 'ARK',
 'ARMADA',
 'ARREIT',
 'ASB',
 'ASDION',
 'FINTEC',
 'ASIABRN',
 'GFM',
 'ASIAFLE',
 'ASIAPAC',
 'ASIAPLY',
 'ASTINO',
 'ASTRO',
 'VIZIONE',
 'AT',
 'ATLAN',
 'ATRIUM',
 'ARBB',
 'AWC',
 'AXIATA',
 'AXREIT',
 'AYS',
 'AZRB',
 'BAHVEST',
 'BARAKAH',
 'BAT',
 'BCB',
 'BDB',
 'BENALEC',
 'BERTAM',
 'BHIC',
 'NGGB',
 'BIG',
 'BIMB',
 'BINTAI',
 'IMPIANA',
 'BIPORT',
 'BJASSET',
 'BAUTO',
 'BJCORP',
 'BJFOOD',
 'BJLAND',
 'BJTOTO',
 'BKAWAN',
 'CHGP',
 'BLDPLNT',
 'BOILERM',
 'BONIA',
 'BORNOIL',
 'BOXPAK',
 'BPPLAS',
 'BPURI',
 'BRAHIMS',
 'BREM',
 'BRIGHT',


In [14]:
len(stock_list)

989

In [15]:
len(stock_name)

989

In [6]:
for stock_code in stock_list:    
    try:     
        
        price_chart_df = get_stock_price(stock_code + ".KL")
        
        close = price_chart_df['Close']
        low = price_chart_df['Low']
        open = price_chart_df['Open']
        high = price_chart_df['High']
        price_chart_df['EMA25'] = add_EMA(close,25)
        price_chart_df['EMA50'] = add_EMA(close,50)
        price_chart_df['EMA100'] = add_EMA(close,100)
        price_chart_df['STOCH_%K(5,3,3)'] = add_STOCH(close, low, high, 5, 3)
        price_chart_df['STOCH_%D(5,3,3)'] = add_STOCH(close, low, high, 5, 3, 3)

        if check_condition(price_chart_df):
            screened_list.append(stock_code)
            
    except Exception as e:
        print(e)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 00EA.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 c

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 29EB.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3021.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3023.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3020.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3019.KL: No data found, symbol may be delisted


In [7]:
screened_list
    
    

['5056', '7043', '5001', '6033', '9873']

## Testing Area

In [8]:
stock_code = "5665"
price_chart_df = get_stock_price(stock_code + ".KL")

open = price_chart_df['Open']
close = price_chart_df['Close']
high = price_chart_df['High']
low = price_chart_df['Low']
price_chart_df['EMA25'] = add_EMA(close,25)
price_chart_df['EMA50'] = add_EMA(close,50)
price_chart_df['EMA100'] = add_EMA(close,100)
price_chart_df['RSI'] = RSIIndicator(close, 14).rsi()



[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [9]:
price_chart_df

,Open,High,Low,Close,Adj Close,Volume,EMA25,EMA50,EMA100,RSI
Date,,,,,,,,,,
2021-01-04,0.840,0.840,0.800,0.800,0.800,1345900,0.800000,0.800000,0.800000,NaN
2021-01-05,0.800,0.820,0.775,0.775,0.775,1209100,0.787000,0.787250,0.787375,NaN
2021-01-06,0.775,0.780,0.700,0.715,0.715,1605700,0.761055,0.762197,0.762766,NaN
2021-01-07,0.710,0.715,0.650,0.670,0.670,1972900,0.735490,0.737747,0.738874,NaN
2021-01-08,0.670,0.690,0.620,0.630,0.630,912600,0.710887,0.714440,0.716220,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-02-28,0.815,0.830,0.815,0.820,0.820,176200,0.816767,0.813855,0.815316,51.383096
2022-03-01,0.810,0.830,0.810,0.820,0.820,1051900,0.817016,0.814096,0.815409,51.383096
2022-03-02,0.820,0.825,0.820,0.820,0.820,155500,0.817245,0.814327,0.815500,51.383096


In [14]:
check_condition(price_chart_df)

False

In [ ]:
stock_list = get_stock_list()

In [14]:
stock_list

['3A',
 'AASIA',
 'AAX',
 'ABFMY1',
 'ABLEGRP',
 'ACME',
 'AXTERIA',
 'ADVENTA',
 'ADVPKG',
 'AEM',
 'AEON',
 'AEONCR',
 'AFFIN',
 'ABMB',
 'AFUJIYA',
 'AHB',
 'AHEALTH',
 'AIM',
 'CAPITALA',
 'AIRPORT',
 'PARKWD',
 'AJI',
 'AJIYA',
 'RGTBHD',
 'ALAM',
 'ALAQAR',
 'ALCOM',
 'ALLIANZ',
 'AMBANK',
 'AMEDIA',
 'AMFIRST',
 'AMTEL',
 'AMWAY',
 'ANALABS',
 'ANCOM',
 'ANCOMLB',
 'ANNJOO',
 'APB',
 'APEX',
 'APM',
 'APOLLO',
 'ARANK',
 'ARK',
 'ARMADA',
 'ARREIT',
 'ASB',
 'ASDION',
 'FINTEC',
 'ASIABRN',
 'GFM',
 'ASIAFLE',
 'ASIAPAC',
 'ASIAPLY',
 'ASTINO',
 'ASTRO',
 'VIZIONE',
 'AT',
 'ATLAN',
 'ATRIUM',
 'ARBB',
 'AWC',
 'AXIATA',
 'AXREIT',
 'AYS',
 'AZRB',
 'BAHVEST',
 'BARAKAH',
 'BAT',
 'BCB',
 'BDB',
 'BENALEC',
 'BERTAM',
 'BHIC',
 'NGGB',
 'BIG',
 'BIMB',
 'BINTAI',
 'IMPIANA',
 'BIPORT',
 'BJASSET',
 'BAUTO',
 'BJCORP',
 'BJFOOD',
 'BJLAND',
 'BJTOTO',
 'BKAWAN',
 'CHGP',
 'BLDPLNT',
 'BOILERM',
 'BONIA',
 'BORNOIL',
 'BOXPAK',
 'BPPLAS',
 'BPURI',
 'BRAHIMS',
 'BREM',
 'BRIGHT',


In [10]:
def check_EMA_crossing(df):
  # condition 1: EMA25 is higher than EMA50 at the last trading day
    cond_1 = df.iloc[-1]['EMA25'] > df.iloc[-1]['EMA50']
  # condition 2: EMA18 is lower than EMA50 the previous day
    cond_2 = df.iloc[-2]['EMA25'] < df.iloc[-2]['EMA50']
  # condition 3: to filter out stocks with less than 50 candles
    cond_3 = len(df.index) > 50 
  # will return True if all 3 conditions are met
    return (cond_1 and cond_2 and cond_3)

In [11]:
def add_EMA(price, day):
    return price.ewm(span=day).mean()


In [12]:
if __name__ == '__main__':
  # a list to store the screened results
    screened_list = [] 
  # get the full stock list
    stock_list = get_stock_list()
    for each_stock in stock_list:
    # Step 1: get stock price for each stock
        price_chart_df = get_stock_price(each_stock)
    # Step 2: add technical indicators (in this case EMA)
        price_chart_df['EMA25']=add_EMA(price_chart_df['Close'],25)
        price_chart_df['EMA50']=add_EMA(price_chart_df['Close'],50)
        price_chart_df['EMA100']=add_EMA(price_chart_df['Close'],100)
    # if all 3 conditions are met, add stock into screened list
        if check_EMA_crossing(price_chart_df):
            screened_list.append(each_stock)
    print(screened_list)

['ASTRO', 'CHOOBEE', 'EG', 'LEBTECH', 'MEDIA', 'OKA', 'PGF', 'PPB', 'DANCO', 'MHCARE', 'OMH']
